## Initial setup

In [1]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [2]:
import numpy
numpy.__version__

'1.26.4'

In [3]:
import os

In [4]:
!pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.3.0
    Uninstalling typeguard-4.3.0:
      Successfully uninstalled typeguard-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.3.1 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [5]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tqdm.notebook import tqdm
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
#import losses
import cv2
from imutils import paths
tf.random.set_seed(666)
np.random.seed(666)

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
import pandas as pd

In [8]:
import os

In [9]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/research/Photocatalysis")

In [10]:
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/research/Photocatalysis'

## Model building

In [11]:
!pip3 install xgboost==1.6.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.1
    Uninstalling xgboost-2.1.1:
      Successfully uninstalled xgboost-2.1.1


In [12]:
from xgboost import XGBRegressor
from tqdm import tqdm
from sklearn.metrics import mean_squared_error # MSE
from sklearn.metrics import mean_absolute_error # MAE
from sklearn.metrics import r2_score # R2

In [13]:
feature_number = 10
feature_number

10

In [14]:
data3 = pd.read_csv('data_after_preprocessing.csv')

In [15]:
data3

,lr_0,lr_1,s_0,s_1,Selected wavelength (nm),Concentration (M),Photocatalyst ratio (mol%),Photocatalyst concentration (M),Reaction flow rate (μL/min),Laser light intensity (mW),non_stable,stable,s_type,lr_type
0,0.0,0.907579,0.000000,0.026131,293.3,0.005,0.5,0.000025,600,772,0.1173,0.0341,1.0,1.0
1,0.0,0.907579,0.000000,0.026131,293.3,0.005,0.5,0.000025,600,1459,0.1600,0.0857,1.0,1.0
2,0.0,0.907579,0.000000,0.026131,293.3,0.005,0.5,0.000025,600,2297,0.2091,0.1558,1.0,1.0
3,0.0,0.907579,0.000000,0.026131,293.3,0.005,0.5,0.000025,600,3082,0.2204,0.2283,1.0,1.0
4,0.0,0.907579,0.000000,0.026131,293.3,0.005,0.5,0.000025,600,3868,0.2890,0.2900,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,1.0,0.000000,0.257762,0.566530,293.3,0.020,2.0,0.000400,200,1459,0.2485,0.2668,5.0,5.0
11996,1.0,0.000000,0.257762,0.566530,293.3,0.020,2.0,0.000400,200,2297,0.4914,0.4493,5.0,5.0
11997,1.0,0.000000,0.257762,0.566530,293.3,0.020,2.0,0.000400,200,3082,0.6098,0.6022,5.0,5.0
11998,1.0,0.000000,0.257762,0.566530,293.3,0.020,2.0,0.000400,200,3868,0.7205,0.6799,5.0,5.0


In [16]:
from itertools import combinations

# Define the full list
full_list = [1, 2, 3, 4, 5]

# Generate all combinations of 2 elements from the full list
combine = list(combinations(full_list, 2))

# For each combination in test_list, find the remaining elements to form the train_list
for element in combine:
    # Using set to find the difference
    train_list = list(set(full_list) - set(element))
    test_list = list(element)
    print(f"Test: {test_list}, Train: {train_list}")

Test: [1, 2], Train: [3, 4, 5]
Test: [1, 3], Train: [2, 4, 5]
Test: [1, 4], Train: [2, 3, 5]
Test: [1, 5], Train: [2, 3, 4]
Test: [2, 3], Train: [1, 4, 5]
Test: [2, 4], Train: [1, 3, 5]
Test: [2, 5], Train: [1, 3, 4]
Test: [3, 4], Train: [1, 2, 5]
Test: [3, 5], Train: [1, 2, 4]
Test: [4, 5], Train: [1, 2, 3]


In [17]:
# Define the full list
full_list = [1, 2, 3, 4, 5]

# Generate all combinations of 2 elements from the full list
combine = list(combinations(full_list, 1))

# For each combination in test_list, find the remaining elements to form the train_list
for element in combine:
    # Using set to find the difference
    train_list = list(set(full_list) - set(element))
    test_list = list(element)
    print(f"Test: {test_list}, Train: {train_list}")

Test: [1], Train: [2, 3, 4, 5]
Test: [2], Train: [1, 3, 4, 5]
Test: [3], Train: [1, 2, 4, 5]
Test: [4], Train: [1, 2, 3, 5]
Test: [5], Train: [1, 2, 3, 4]


### cross photocatalyst prediction

In [18]:
import os
import pandas as pd
import numpy as np
from itertools import combinations
from openpyxl import Workbook
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm

# Define the full list
full_list = [1, 2, 3, 4, 5]

# Generate all combinations of 2 elements from the full list
combine = list(combinations(full_list, 2))

# Create a list to store results
results = []

# For each combination in test_list, find the remaining elements to form the train_list
for element in combine:
    # Using set to find the difference
    train_list = list(set(full_list) - set(element))
    test_list = list(element)
    print(f"Test: {test_list}, Train: {train_list}")

    X_train = data3[data3['lr_type'].isin(train_list)].iloc[:, 0:feature_number]
    X_test = data3[data3['lr_type'].isin(test_list)].iloc[:, 0:feature_number]

    y_train = data3[data3['lr_type'].isin(train_list)].iloc[:, feature_number:feature_number+1]
    y_test = data3[data3['lr_type'].isin(test_list)].iloc[:, feature_number:feature_number+1]

    for j in tqdm(range(1)):
        regressor = XGBRegressor()
        regressor.fit(X_train, y_train)

        y_pred = regressor.predict(X_test)
        df = pd.DataFrame(y_test)
        df['y_pred'] = y_pred
        df.columns = ['y_test', 'y_pred']

    y_test = df['y_test']
    y_predict = df['y_pred']

    # Calculate metrics
    mse = mean_squared_error(y_test, y_predict)
    mae = mean_absolute_error(y_test, y_predict)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_predict)

    # Print metrics
    print(mse)
    print(mae)
    print(rmse)
    print(r2)

    # Store results
    results.append([test_list, mae, rmse])

    output = data3[data3['lr_type'].isin(test_list)]
    output['predict'] = df['y_pred']

    file_path = r"photocatalyst_3predict2_0704.xlsx"

    if not os.path.exists(file_path):
        wb = Workbook()
        wb.save(file_path)

    work = pd.ExcelWriter(file_path, mode="a", engine="openpyxl")

    #output.to_excel(work, sheet_name=str(test_list[0]) + '&' + str(test_list[1]), index=False)

    work._save()

# Save results to CSV file
results_df = pd.DataFrame(results, columns=['test_list', 'MAE', 'RMSE'])
#results_df.to_csv('photocatalyst_result_3predict2_0704.csv', index=False)

Test: [1, 2], Train: [3, 4, 5]


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:61: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))
100%|██████████| 1/1 [00:02<00:00,  2.24s/it]
<ipython-input-18-3ae1a9f0d0f2>:60: SettingWithCopyWarning: 
A value is trying to 

0.005986018414170313
0.0576773264756108
0.07736936353732214
0.863941815494994
Test: [1, 3], Train: [2, 4, 5]


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:61: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Function "cuInit" not found
  warnings.warn(str(e))
100%|██████████| 1/1 [00:03<00:00,  3.67s/it]
<ipython-input-18-3ae1a9f0d0f2>:60: SettingWithCopyWarning: 
A value is trying to be

0.008068953478198878
0.066696845393242
0.08982735373035809
0.8136575836456572
Test: [1, 4], Train: [2, 3, 5]


...

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:61: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Function "cuInit" not found
  warnings.warn(str(e))
100%|██████████| 1/1 [00:01<00:00,  1.41s/it]
<ipython-input-18-3ae1a9f0d0f2>:60: SettingWithCopyWarning: 
A value is trying to be

0.012915596902328418
0.07625030632956602
0.11364680770848083
0.6854593173338623
Test: [4, 5], Train: [1, 2, 3]


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:61: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Function "cuInit" not found
  warnings.warn(str(e))
100%|██████████| 1/1 [00:00<00:00,  1.07it/s]

0.013317564436408176
0.08613431839823493
0.11540175231082142
0.6466304850996308



<ipython-input-18-3ae1a9f0d0f2>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['predict'] = df['y_pred']


In [19]:
work.close()

In [20]:
# Define the full list
full_list = [1, 2, 3, 4, 5]

# Generate all combinations of 2 elements from the full list
combine = list(combinations(full_list, 1))

# Create a list to store results
results = []

# For each combination in test_list, find the remaining elements to form the train_list
for element in combine:
    # Using set to find the difference
    train_list = list(set(full_list) - set(element))
    test_list = list(element)
    print(f"Test: {test_list}, Train: {train_list}")

    X_train = data3[data3['lr_type'].isin(train_list)].iloc[:, 0:feature_number]
    X_test = data3[data3['lr_type'].isin(test_list)].iloc[:, 0:feature_number]

    y_train = data3[data3['lr_type'].isin(train_list)].iloc[:, feature_number:feature_number+1]
    y_test = data3[data3['lr_type'].isin(test_list)].iloc[:, feature_number:feature_number+1]

    for j in tqdm(range(1)):
        regressor = XGBRegressor()
        regressor.fit(X_train, y_train)

        y_pred = regressor.predict(X_test)
        df = pd.DataFrame(y_test)
        df['y_pred'] = y_pred
        df.columns = ['y_test', 'y_pred']

    y_test = df['y_test']
    y_predict = df['y_pred']

    # Calculate metrics
    mse = mean_squared_error(y_test, y_predict)
    mae = mean_absolute_error(y_test, y_predict)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_predict)

    # Print metrics
    print(mse)
    print(mae)
    print(rmse)
    print(r2)

    # Store results
    results.append([test_list, mae, rmse])

    output = data3[data3['lr_type'].isin(test_list)]
    output['predict'] = df['y_pred']

    file_path = r"photocatalyst_4predict1_0704.xlsx"

    if not os.path.exists(file_path):
        wb = Workbook()
        wb.save(file_path)

    work = pd.ExcelWriter(file_path, mode="a", engine="openpyxl")

    #output.to_excel(work, sheet_name=str(test_list[0]), index=False)

    work._save()

# Save results to CSV file
results_df = pd.DataFrame(results, columns=['test_list', 'MAE', 'RMSE'])
#results_df.to_csv('photocatalyst_result_4predict1_0704.csv', index=False)

Test: [1], Train: [2, 3, 4, 5]


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:61: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Function "cuInit" not found
  warnings.warn(str(e))
100%|██████████| 1/1 [00:00<00:00,  1.29it/s]
<ipython-input-20-e1a2b1255b6b>:51: SettingWithCopyWarning: 
A value is trying to be

0.014173528730062038
0.09997520837238572
0.11905263008460602
0.7092326429574876
Test: [2], Train: [1, 3, 4, 5]


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:61: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Function "cuInit" not found
  warnings.warn(str(e))
100%|██████████| 1/1 [00:00<00:00,  1.31it/s]
<ipython-input-20-e1a2b1255b6b>:51: SettingWithCopyWarning: 
A value is trying to be

0.001843270392501801
0.03321157480051272
0.04293332496443527
0.9473828305811016
Test: [3], Train: [1, 2, 4, 5]


...

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:61: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Function "cuInit" not found
  warnings.warn(str(e))
100%|██████████| 1/1 [00:00<00:00,  1.31it/s]
<ipython-input-20-e1a2b1255b6b>:51: SettingWithCopyWarning: 
A value is trying to be

0.011786674930007084
0.08368579258560871
0.10856645398099306
0.4806386486909181
Test: [5], Train: [1, 2, 3, 4]


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:61: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Function "cuInit" not found
  warnings.warn(str(e))
100%|██████████| 1/1 [00:00<00:00,  1.30it/s]

0.02370976639172407
0.11833873433570873
0.1539797596819922
0.48544134021680774



<ipython-input-20-e1a2b1255b6b>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['predict'] = df['y_pred']


In [21]:
work.close()

### cross substrate prediction

In [22]:
import os
import pandas as pd
import numpy as np
from itertools import combinations
from openpyxl import Workbook
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define the full list
full_list = [1, 2, 4, 5]

# Generate all combinations of 2 elements from the full list
combine = list(combinations(full_list, 2))

# Create a list to store results
results = []

# For each combination in test_list, find the remaining elements to form the train_list
for element in combine:
    # Using set to find the difference
    train_list = list(set(full_list) - set(element))
    test_list = list(element)
    print(f"Test: {test_list}, Train: {train_list}")

    X_train = data3[data3['s_type'].isin(train_list)].iloc[:, 0:feature_number]
    X_test = data3[data3['s_type'].isin(test_list)].iloc[:, 0:feature_number]

    y_train = data3[data3['s_type'].isin(train_list)].iloc[:, feature_number:feature_number+1]
    y_test = data3[data3['s_type'].isin(test_list)].iloc[:, feature_number:feature_number+1]

    for j in range(1):
        regressor = XGBRegressor()
        regressor.fit(X_train, y_train)

        y_pred = regressor.predict(X_test)
        df = pd.DataFrame(y_test)
        df['y_pred'] = y_pred
        df.columns = ['y_test', 'y_pred']

    y_test = df['y_test']
    y_predict = df['y_pred']

    # Calculate metrics
    mse = mean_squared_error(y_test, y_predict)
    mae = mean_absolute_error(y_test, y_predict)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_predict)

    # Print metrics
    print(mse)
    print(mae)
    print(rmse)
    print(r2)

    # Store results
    results.append([test_list, mae, rmse])

    output = data3[data3['s_type'].isin(test_list)]
    output['predict'] = df['y_pred']

    file_path = r"substrate_2predict2_0704.xlsx"

    if not os.path.exists(file_path):
        wb = Workbook()
        wb.save(file_path)

    work = pd.ExcelWriter(file_path, mode="a", engine="openpyxl")

    #output.to_excel(work, sheet_name=str(test_list[0]) + '&' + str(test_list[1]), index=False)

    work._save()

# Save results to CSV file
results_df = pd.DataFrame(results, columns=['test_list', 'MAE', 'RMSE'])
#results_df.to_csv('substrate_result_2predict2_0704.csv', index=False)

Test: [1, 2], Train: [4, 5]


/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:61: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Function "cuInit" not found
  warnings.warn(str(e))


0.010000776919960987
0.07733107517473412
0.10000388452435728
0.786042262528418
Test: [1, 4], Train: [2, 5]


<ipython-input-22-78175e20b8df>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['predict'] = df['y_pred']
/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:61: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library

0.005899162886377049
0.05837738401967463
0.07680600813983922
0.8350904919919157
Test: [1, 5], Train: [2, 4]


...

<ipython-input-22-78175e20b8df>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['predict'] = df['y_pred']
/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:61: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library

0.009973175175236909
0.07735981172597256
0.09986578580893914
0.7689943026425748
Test: [4, 5], Train: [1, 2]


<ipython-input-22-78175e20b8df>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['predict'] = df['y_pred']
/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:61: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library

0.009974404825644536
0.07722040237887239
0.09987194213413764
0.6980509519214112


<ipython-input-22-78175e20b8df>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['predict'] = df['y_pred']


In [23]:
work.close()

In [24]:
# Define the full list
full_list = [1, 2, 4, 5]

# Generate all combinations of 2 elements from the full list
combine = list(combinations(full_list, 1))

# Create a list to store results
results = []

# For each combination in test_list, find the remaining elements to form the train_list
for element in combine:
    # Using set to find the difference
    train_list = list(set(full_list) - set(element))
    test_list = list(element)
    print(f"Test: {test_list}, Train: {train_list}")

    X_train = data3[data3['s_type'].isin(train_list)].iloc[:, 0:feature_number]
    X_test = data3[data3['s_type'].isin(test_list)].iloc[:, 0:feature_number]

    y_train = data3[data3['s_type'].isin(train_list)].iloc[:, feature_number:feature_number+1]
    y_test = data3[data3['s_type'].isin(test_list)].iloc[:, feature_number:feature_number+1]

    for j in range(1):
        regressor = XGBRegressor()
        regressor.fit(X_train, y_train)

        y_pred = regressor.predict(X_test)
        df = pd.DataFrame(y_test)
        df['y_pred'] = y_pred
        df.columns = ['y_test', 'y_pred']

    y_test = df['y_test']
    y_predict = df['y_pred']

    # Calculate metrics
    mse = mean_squared_error(y_test, y_predict)
    mae = mean_absolute_error(y_test, y_predict)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_predict)

    # Print metrics
    print(mse)
    print(mae)
    print(rmse)
    print(r2)

    # Store results
    results.append([test_list, mae, rmse])

    output = data3[data3['s_type'].isin(test_list)]
    output['predict'] = df['y_pred']


    file_path = r"substrate_3predict1_0704.xlsx"

    if not os.path.exists(file_path):
        wb = Workbook()
        wb.save(file_path)

    work = pd.ExcelWriter(file_path, mode="a", engine="openpyxl")

    output.to_excel(work, sheet_name=str(test_list[0]), index=False)

    work._save()

# Save results to CSV file
results_df = pd.DataFrame(results, columns=['test_list', 'MAE', 'RMSE'])
#results_df.to_csv('substrate_result_3predict1_0704.csv', index=False)

Test: [1], Train: [2, 4, 5]


/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:61: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Function "cuInit" not found
  warnings.warn(str(e))


0.007848304417855211
0.06580804248528989
0.08859065649296889
0.820748052232176


<ipython-input-24-5fca2a6e2e0f>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['predict'] = df['y_pred']


Test: [2], Train: [1, 4, 5]


...

<ipython-input-24-5fca2a6e2e0f>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['predict'] = df['y_pred']


Test: [5], Train: [1, 2, 4]


/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:61: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Function "cuInit" not found
  warnings.warn(str(e))


0.007710155729917469
0.06982887287558931
0.08780749244749829
0.798852960445183


<ipython-input-24-5fca2a6e2e0f>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['predict'] = df['y_pred']


In [25]:
work.close()